In [1]:
import xarray as xr
from dask.distributed import Client
import numpy as np

In [2]:
client = Client(n_workers=7, threads_per_worker=1, memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:55712 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 7 Memory: 14.00 GB


In [3]:
xrast = xr.open_rasterio('C:/Users/Eric W/Downloads/Soil_NLCD_Geo_Imperv_Use/Soil_NLCD_Geo_Imperv_Use.tif',chunks={'x': 512, 'y': 512, 'band':1}).squeeze()

In [4]:
rmp = np.genfromtxt('C:/Projects/KingCounty/3_Data/HRU_Unique_Vals_Remap.csv',delimiter=',')

In [5]:
testval = rmp[0,1]

In [ ]:
remaptargs = np.unique(rmp[:,1])
def remap(array):
    if sum(array.shape) == 0:
        return(array)
    array = array.stack(z=('x','y'))
    for val in remaptargs:
        toremap = rmp[np.where(rmp[:,1]==val),0]
        array[np.in1d(array, toremap)] = val 
    array = array.unstack('z')
    return(array)

In [6]:
def single_remap(array, newval):
    if sum(array.shape) == 0:
        return(array)
    array = array.stack(z=('x','y'))
    toremap = rmp[np.where(rmp[:,1]== newval),0]
    array[np.in1d(array, toremap)] = newval
    array = array.unstack('z')
    return(array)

In [7]:
remap = xr.map_blocks(func=single_remap,obj=xrast,kwargs={'newval':testval})

In [ ]:
remap_vals = remap.compute()

In [ ]:
start = 25600
nchunks = 10

In [ ]:
subarr = xrast[start:(start+nchunks*512),start:(start+nchunks*512)]
print('Array is ' + str(((subarr.shape[0]*subarr.shape[1]*32)/8)/1000000) + ' megabytes.')

In [ ]:
subarr = subarr.compute()

In [ ]:
submap = xr.map_blocks(func=remap,obj=subarr)